In [1]:
import sys
sys.path.insert(0, '../src/')

In [2]:
from data.data_load import DataLoad
from data.data_validation import DataValidation

In [3]:
dl = DataLoad()
dv = DataValidation()

In [4]:
df = dl.load_data('train_dataset_name')

2023-12-02 17:27:43 [info     ] Iniciando o carregamento


In [5]:
df

,target,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
149995,0,0.040674,74,0,0.225131,2100.0,4,0,1,0,0.0
149996,0,0.299745,44,0,0.716562,5584.0,4,0,1,0,2.0
149997,0,0.246044,58,0,3870.000000,NaN,18,0,1,0,0.0
149998,0,0.000000,30,0,0.000000,5716.0,4,0,0,0,0.0


In [6]:
is_valid = dv.run(df)

2023-12-02 17:27:43 [info     ] Validação iniciou..
2023-12-02 17:27:43 [info     ] Validation columns passed...
2023-12-02 17:27:43 [info     ] Validacao com sucesso.


In [7]:
from data.data_transformation import DataTransformation

In [8]:
dt = DataTransformation(df)

In [9]:
X_train, X_valid, y_train, y_valid = dt.train_test_split()

In [10]:
X_train.shape

(100500, 10)

In [11]:
X_valid.shape

(49500, 10)

In [12]:
from sklearn.pipeline import Pipeline
from feature_engine.discretisation import EqualFrequencyDiscretiser, EqualWidthDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.wrappers import SklearnTransformerWrapper
from sklearn.preprocessing import RobustScaler, StandardScaler

In [13]:
from data.data_preprocess import DataPreprocess
from utils.utils import load_config_file

In [16]:
pipe = Pipeline([
    ('imputer', MeanMedianImputer(variables=load_config_file().get('vars_imputer'))),
    ('discretizer', EqualFrequencyDiscretiser(variables=load_config_file().get('vars_discretizer'))),
    ('scaler', SklearnTransformerWrapper(StandardScaler()))
])

In [17]:
dp = DataPreprocess(pipe)

In [18]:
dp.train(X_train)

2023-12-02 17:30:29 [info     ] Iniciando o processamento


Pipeline(steps=[('imputer',
                 MeanMedianImputer(variables=['RendaMensal',
                                              'NumeroDeDependentes'])),
                ('discretizer',
                 EqualFrequencyDiscretiser(variables=['TaxaDeUtilizacaoDeLinhasNaoGarantidas',
                                                      'TaxaDeEndividamento',
                                                      'RendaMensal'])),
                ('scaler',
                 SklearnTransformerWrapper(transformer=StandardScaler()))])

In [19]:
X_train_processed = dp.transform(X_train)
X_valid_processed = dp.transform(X_valid)

2023-12-02 17:31:03 [info     ] Iniciando a Transformação
2023-12-02 17:31:04 [info     ] Iniciando a Transformação


In [20]:
X_train_processed

,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
146433,0.174078,-1.507791,-0.100473,-0.870389,0.558077,-0.672878,-0.063940,0.869889,-0.057852,0.236830
15597,0.870388,-0.830642,-0.100473,-0.870389,-1.184396,-1.255080,-0.063940,-0.902282,-0.057852,-0.667136
111605,1.566699,-1.236931,-0.100473,-1.218546,-0.487407,-1.449147,-0.063940,-0.902282,-0.057852,-0.667136
85418,-1.218544,2.081102,-0.100473,1.218553,-0.138912,-0.866945,-0.063940,-0.902282,-0.057852,-0.667136
9652,0.870388,0.591373,-0.100473,-0.870389,-0.487407,-1.061013,-0.063940,-0.902282,-0.057852,1.140796
...,...,...,...,...,...,...,...,...,...,...
90431,-1.566699,-0.288922,-0.100473,0.522239,1.603560,0.879659,-0.063940,4.414231,-0.057852,-0.667136
18048,-1.218544,-0.762927,-0.100473,0.522239,0.906571,0.103390,-0.063940,0.869889,-0.057852,-0.667136
3895,-1.566699,-0.221207,-0.100473,-0.174075,1.255066,1.849995,-0.063940,-0.016196,-0.057852,0.236830
74354,1.218544,0.726803,-0.100473,0.174082,-1.532890,-1.255080,0.176004,-0.902282,-0.057852,-0.667136


In [21]:
X_valid_processed

,TaxaDeUtilizacaoDeLinhasNaoGarantidas,Idade,NumeroDeVezes30-59DiasAtrasoNaoPior,TaxaDeEndividamento,RendaMensal,NumeroDeLinhasDeCreditoEEmprestimosAbertos,NumeroDeVezes90DiasAtraso,NumeroDeEmprestimosOuLinhasImobiliarias,NumeroDeVezes60-89DiasAtrasoNaoPior,NumeroDeDependentes
118607,0.522233,0.185083,-0.100473,0.870396,0.558077,0.491525,-0.063940,1.755975,-0.057852,0.236830
112706,0.870388,-1.169216,-0.100473,-1.218546,0.906571,-0.672878,-0.063940,-0.902282,-0.057852,0.236830
57891,-1.566699,-1.575506,-0.100473,-0.870389,-1.184396,-0.478811,-0.063940,-0.902282,-0.057852,-0.667136
13081,0.870388,0.049653,-0.100473,-0.870389,-0.487407,0.297458,-0.063940,-0.902282,-0.057852,-0.667136
141317,-1.218544,-0.356637,-0.100473,1.566710,-0.138912,-1.061013,-0.063940,-0.016196,-0.057852,-0.667136
...,...,...,...,...,...,...,...,...,...,...
103450,-0.522233,-1.169216,0.138104,1.218553,-0.138912,-1.061013,-0.063940,-0.902282,-0.057852,-0.667136
54368,1.566699,-1.033786,0.138104,-0.870389,-0.835901,-0.672878,0.895834,-0.902282,0.664426,-0.667136
144666,-1.566699,-0.830642,-0.100473,1.566710,-0.138912,0.297458,-0.063940,0.869889,-0.057852,-0.667136
97933,-1.218544,-0.221207,-0.100473,-0.522232,1.603560,-0.478811,-0.063940,2.642060,-0.057852,-0.667136
